<a href="https://colab.research.google.com/github/halylo/BaoCao_THTTNT/blob/main/TimKiemDoiKhang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TIC-TAC-TOE ALPHA-BEta PRUNING

In [21]:
%%writefile core.py
# core.py - Logic Trò chơi và AI (Alpha-Beta Pruning có Heuristic)

import copy
import math
import random

# Định nghĩa các hằng số
X = "X"
O = "O"
EMPTY = " "

class TicTacToeNXN:
    """
    Lớp chứa logic cốt lõi của trò chơi Tic-Tac-Toe NxN,
    bao gồm các thuật toán Alpha-Beta Pruning.
    """
    def __init__(self, n=3, user=None, ai=None):
        self.n = n
        self.board = [[EMPTY for _ in range(n)] for _ in range(n)]
        self.user = user
        self.ai = ai

    def copy(self):
        new = TicTacToeNXN(self.n, self.user, self.ai)
        new.board = copy.deepcopy(self.board)
        return new

    def player_turn(self):
        """Xác định lượt chơi hiện tại (X hoặc O). X luôn đi trước."""
        count = sum(1 for row in self.board for cell in row if cell != EMPTY)
        return X if count % 2 == 0 else O

    def actions(self):
        """Trả về danh sách các nước đi hợp lệ (vị trí trống)."""
        return [(i, j) for i in range(self.n) for j in range(self.n) if self.board[i][j] == EMPTY]

    def result(self, action):
        """Trả về trạng thái trò chơi mới sau khi thực hiện nước đi."""
        game = self.copy()
        i, j = action
        game.board[i][j] = self.player_turn()
        return game

    def winner(self):
        """Kiểm tra và trả về người chiến thắng (X hoặc O) nếu có."""
        # Kiểm tra hàng, cột, đường chéo (Full N liên tiếp để thắng)
        for i in range(self.n):
            if self.board[i][0] != EMPTY and all(self.board[i][j] == self.board[i][0] for j in range(self.n)):
                return self.board[i][0]
        for j in range(self.n):
            if self.board[0][j] != EMPTY and all(self.board[i][j] == self.board[0][j] for i in range(self.n)):
                return self.board[0][j]

        # Đường chéo chính
        if self.board[0][0] != EMPTY and all(self.board[i][i] == self.board[0][0] for i in range(self.n)):
            return self.board[0][0]

        # Đường chéo phụ
        if self.board[0][self.n-1] != EMPTY and all(self.board[i][self.n-1-i] == self.board[0][self.n-1] for i in range(self.n)):
            return self.board[0][self.n-1]

        return None

    def terminal(self):
        """Kiểm tra trạng thái kết thúc trò chơi."""
        return self.winner() is not None or not self.actions()

    def utility(self):
        """Tính giá trị tiện ích cho trạng thái cuối (AI thắng: 1, User thắng: -1, Hòa: 0)."""
        win = self.winner()
        if win == self.ai: return 1
        if win == self.user: return -1
        return 0

    # === Hàm Heuristic để đánh giá trạng thái không kết thúc ===
    def evaluate(self):
        """
        Hàm Heuristic: Đánh giá sức mạnh của bảng hiện tại (non-terminal state).
        Điểm thưởng/phạt cho các hàng/cột/đường chéo gần thắng (N-1, N-2 quân).
        """
        score = 0
        lines = []

        # Lấy tất cả các đường (hàng, cột, đường chéo)
        for r in range(self.n):
            lines.append([self.board[r][c] for c in range(self.n)])
        for c in range(self.n):
            lines.append([self.board[r][c] for r in range(self.n)])
        lines.append([self.board[i][i] for i in range(self.n)]) # Chéo chính
        lines.append([self.board[i][self.n-1-i] for i in range(self.n)]) # Chéo phụ

        for line in lines:
            ai_count = line.count(self.ai)
            user_count = line.count(self.user)
            empty_count = line.count(EMPTY)

            # Bỏ qua các đường đã bị chặn hoàn toàn
            if ai_count > 0 and user_count > 0:
                continue

            # Đánh giá cho AI (Maximizing Player)
            if ai_count > 0:
                if ai_count == self.n - 1 and empty_count >= 1:
                    score += 1000  # Ngay lập tức thắng
                elif ai_count == self.n - 2 and empty_count >= 2:
                    score += 50   # Tạo thế 2 nước nữa thắng
                elif ai_count == 1 and empty_count == self.n - 1:
                    score += 1

            # Đánh giá cho Người chơi (Minimizing Player) - Cần chặn
            if user_count > 0:
                if user_count == self.n - 1 and empty_count >= 1:
                    score -= 1000 # Cần chặn gấp
                elif user_count == self.n - 2 and empty_count >= 2:
                    score -= 50   # Khả năng người chơi sắp thắng

        return score

    # === Alpha-Beta Giới hạn độ sâu (Sử dụng Heuristic) ===

    def alpha_beta_limited(self, depth, cur_depth=0, alpha=-math.inf, beta=math.inf, maximizing=True):
        if self.terminal():
            # Trạng thái kết thúc: Trả về utility với giá trị rất lớn để ưu tiên tuyệt đối
            return self.utility() * 1000000

        if cur_depth >= depth:
            # Đạt giới hạn độ sâu: Trả về Heuristic
            return self.evaluate()

        if maximizing:
            max_eval = -math.inf
            for a in self.actions():
                score = self.result(a).alpha_beta_limited(depth, cur_depth+1, alpha, beta, False)
                max_eval = max(max_eval, score)
                alpha = max(alpha, max_eval)
                if beta <= alpha: break
            return max_eval
        else:
            min_eval = math.inf
            for a in self.actions():
                score = self.result(a).alpha_beta_limited(depth, cur_depth+1, alpha, beta, True)
                min_eval = min(min_eval, score)
                beta = min(beta, min_eval)
                if beta <= alpha: break
            return min_eval

    # === Hàm Alpha-Beta Full đã bị loại bỏ vì quá chậm, chỉ giữ lại để tham khảo (nếu cần) ===
    def alpha_beta_full(self, alpha=-math.inf, beta=math.inf, maximizing=True):
        # KHÔNG SỬ DỤNG HÀM NÀY cho N >= 4. Giữ lại nếu N=3
        if self.terminal():
            return self.utility() * 1000000

        # ... (code Alpha-Beta Full) ...
        pass


    def get_best_move(self, difficulty):
        """Tính toán và trả về nước đi tối ưu cho AI dựa trên độ khó."""
        if difficulty == "Dễ":
            return random.choice(self.actions()) if self.actions() else None

        maximizing_player = (self.player_turn() == self.ai)
        best_val = -math.inf
        best_move = None

        # Thiết lập giới hạn độ sâu (Depth Limit) cho Heuristic Search
        if difficulty == "Trung bình":
            depth_limit = 4
        elif difficulty == "Khó":
            # Tăng độ sâu cho độ khó cao, cân bằng giữa tốc độ và độ sâu chiến thuật
            depth_limit = 6

        if self.n > 4 and difficulty == "Khó":
             # Giảm độ sâu nếu kích thước quá lớn để tránh timeout
            depth_limit = 4

        if not self.actions():
            return None

        # Sử dụng Alpha-Beta giới hạn cho cả 2 độ khó (trừ Dễ)
        for a in self.actions():
            val = self.result(a).alpha_beta_limited(depth_limit, 1, maximizing=not maximizing_player)

            # Cập nhật nước đi tốt nhất
            if val > best_val or best_move is None:
                best_val = val
                best_move = a
        return best_move

Overwriting core.py


In [22]:
# helper.py
%%writefile helper.py
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from core import X, O, EMPTY, TicTacToeNXN # Import các hằng số và lớp từ core.py

# === KHỞI TẠO CÁC WIDGET CHUNG ===

def initialize_widgets():
    """Khởi tạo và trả về các widget chính cho giao diện."""
    display(HTML("""
    <style>
    .widget-slider, .widget-radio-buttons, .widget-dropdown {
        background-color: #FFF9C4 !important;
        border: 2px solid #FFECB3 !important;
        border-radius: 12px !important;
        padding: 5px !important;
        margin: 5px 0 !important;
    }
    .widget-label { color: #333 !important; font-weight: bold !important; }
    </style>
    """))

    out = widgets.Output()
    status_label = widgets.Label(value="🎮 Chào mừng đến với Tic-Tac-Toe NxN!", style={'font_weight': 'bold', 'font_size': '18px'})
    grid_container = widgets.Output()

    n_slider = widgets.IntSlider(value=3, min=3, max=5, step=1, description='Kích thước N:', description_width='120px')
    player_choice = widgets.RadioButtons(options=[('X - Bạn đi trước', X), ('O - AI đi trước', O)], description='Bạn chơi:', description_width='120px')
    difficulty_choice = widgets.Dropdown(
        options=[('Dễ 🟢', 'Dễ'), ('Trung bình 🟡', 'Trung bình'), ('Khó 🔴', 'Khó')],
        value='Khó',
        description='Độ khó AI:', description_width='120px'
    )

    start_button = widgets.Button(description="🚀 Bắt đầu chơi", button_style='success',
                                  layout=widgets.Layout(width='240px', height='60px'))
    restart_button = widgets.Button(description="🔄 Chơi lại", button_style='warning',
                                    layout=widgets.Layout(width='160px', height='50px'))

    return n_slider, player_choice, difficulty_choice, start_button, restart_button, out, status_label, grid_container

# === HÀM VẼ BẢNG VÀ CẬP NHẬT GIAO DIỆN ===

def create_board_ui(n, player_click_handler):
    """Tạo GridBox chứa các nút bấm cho bảng trò chơi."""
    buttons = []
    grid = widgets.GridBox(layout=widgets.Layout(
        grid_template_columns=f"repeat({n}, 120px)",
        grid_gap="12px",
        justify_content="center",
        padding="20px"
    ))
    for i in range(n):
        for j in range(n):
            btn = widgets.Button(
                description="",
                layout=widgets.Layout(width='115px', height='115px'),
                style={'font_size': '52px', 'font_weight': 'bold', 'button_color': '#ffffff'}
            )
            # Gán hàm xử lý sự kiện click
            btn.on_click(lambda b, r=i, c=j: player_click_handler(r, c))
            buttons.append(btn)
            grid.children += (btn,)
    return grid, buttons

def update_board_ui(game, buttons):
    """Cập nhật trạng thái các nút bấm trên giao diện dựa trên trạng thái trò chơi."""
    for idx, btn in enumerate(buttons):
        i, j = divmod(idx, game.n)
        val = game.board[i][j]
        btn.description = val if val != EMPTY else ""
        btn.style.text_color = '#0d6efd' if val == X else '#dc3545' if val == O else '#666'
        btn.style.button_color = '#FFF9C4' if val != EMPTY else '#FFF9C4'
        btn.disabled = (val != EMPTY or game.terminal())

def display_game_end(game, out, status_label):
    """Hiển thị thông báo kết thúc trò chơi (thắng/hòa)."""
    win = game.winner()
    if win:
        winner = "Bạn" if win == game.user else "AI"
        color = "green" if win == game.user else "red"
        status_label.value = f"🎉 {winner} thắng!"
        msg = f"<h2 style='color:{color}; text-align:center; font-size:36px'>🎉 {winner} THẮNG!</h2>"
    else:
        status_label.value = "🤝 Hòa!"
        msg = "<h2 style='color:#fd7e14; text-align:center; font-size:36px'>🤝 HÒA!</h2>"
    with out:
        clear_output()
        display(HTML(msg))

def display_initial_ui(n_slider, player_choice, difficulty_choice, start_button, restart_button, status_label, grid_container, out):
    """Hiển thị toàn bộ giao diện trò chơi."""
    display(HTML("<h1 style='text-align:center; color:#1976D2; font-family:Arial Black'>🎮 TIC-TAC-TOE</h1>"))
    display(HTML("<p style='text-align:center; font-size:18px; color:#666'> Alpha-Beta Pruning ✨</p>"))

    display(widgets.VBox([
        widgets.HBox([n_slider, player_choice, difficulty_choice], layout=widgets.Layout(justify_content='center', margin='15px', align_items='center')),
        widgets.HBox([start_button], layout=widgets.Layout(justify_content='center')),
        status_label,
        grid_container,
        out,
        widgets.HBox([restart_button], layout=widgets.Layout(justify_content='center', margin='20px 0 10px 0'))
    ]))

Overwriting helper.py


In [23]:
# main.py
%%writefile main.py
from core import X, O, TicTacToeNXN, EMPTY
from helper import (
    initialize_widgets, create_board_ui, update_board_ui,
    display_game_end, display_initial_ui
)
from IPython.display import clear_output, HTML

# === BIẾN TOÀN CỤC ===
game: TicTacToeNXN = None
buttons = []
current_difficulty = "Khó"

# === KHỞI TẠO WIDGETS ===
(n_slider, player_choice, difficulty_choice, start_button, restart_button,
 out, status_label, grid_container) = initialize_widgets()

# === HÀM XỬ LÝ SỰ KIỆN ===

def handle_player_click(row, col):
    """Xử lý sự kiện khi người dùng click vào một ô trên bảng."""
    global game
    if not game or game.terminal() or game.player_turn() != game.user or game.board[row][col] != EMPTY:
        return

    # 1. Người dùng đi
    game.board[row][col] = game.user
    update_board_ui(game, buttons)

    if game.terminal():
        display_game_end(game, out, status_label)
        return

    # 2. Lượt AI
    handle_ai_turn()

def handle_ai_turn():
    """Xử lý lượt đi của AI."""
    global game, current_difficulty

    status_label.value = f"🤖 AI ({current_difficulty}) đang suy nghĩ..."
    with out:
        clear_output(wait=True)
        display(HTML("<i>Đang tính nước đi tối ưu...</i>"))

    move = game.get_best_move(current_difficulty)

    if move:
        game.board[move[0]][move[1]] = game.ai
        update_board_ui(game, buttons)

        if game.terminal():
            display_game_end(game, out, status_label)
        else:
            status_label.value = f"🟢 Lượt của bạn ({game.user})"
            with out:
                clear_output()
    else:
        # Xử lý trường hợp không còn nước đi (chỉ xảy ra khi trò chơi đã kết thúc)
        if game.terminal():
             display_game_end(game, out, status_label)

def start_game(b):
    """Xử lý sự kiện khi click nút Bắt đầu chơi."""
    global game, current_difficulty, buttons
    n = n_slider.value
    user_sym = player_choice.value
    current_difficulty = difficulty_choice.value

    # AI là ký hiệu còn lại
    ai_sym = O if user_sym == X else X

    game = TicTacToeNXN(n, user_sym, ai_sym)

    with grid_container:
        clear_output(wait=True)
        # Tạo bảng giao diện và lưu trữ các nút bấm
        grid, buttons = create_board_ui(n, handle_player_click)
        display(grid)

    update_board_ui(game, buttons)
    status_label.value = f"📏 {n}x{n} | Bạn: {game.user} | AI: {game.ai} | Độ khó: {current_difficulty} | Lượt: {game.player_turn()}"
    with out:
        clear_output()

    # Nếu là lượt AI → tự động đánh ngay
    if game.player_turn() == game.ai:
        handle_ai_turn()

def reset_game(b):
    """Xử lý sự kiện khi click nút Chơi lại."""
    global game, buttons
    game = None
    buttons = []
    with grid_container:
        clear_output(wait=True)
    with out:
        clear_output(wait=True)
    status_label.value = "🎮 Chào mừng đến với Tic-Tac-Toe NxN!"

# === GẮN HÀM XỬ LÝ VÀ HIỂN THỊ CUỐI CÙNG ===
start_button.on_click(start_game)
restart_button.on_click(reset_game)

if __name__ == '__main__':
    # Hiển thị giao diện chính trong Colab/Jupyter Notebook
    display_initial_ui(n_slider, player_choice, difficulty_choice, start_button, restart_button, status_label, grid_container, out)

Overwriting main.py


In [24]:
%run main.py